In [0]:
pip install azure-eventhub faker

In [0]:
import json
import time
import uuid
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from faker import Faker
import random

# -------------------
# Azure Event Hub Config
# -------------------
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-customer-policy")
EVENTHUB_NAME = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-customer-hub")

# -------------------
# Faker Setup
# -------------------
fake = Faker("en_IN")  # Indian locale

def custom_indian_phone_number():
    # Choose starting digit from 6, 7, 8, 9
    start_digit = str(random.choice([6, 7, 8, 9]))
    # Generate the remaining 9 digits
    remaining = fake.numerify('#########')
    return start_digit + remaining


def generate_customer(customer_id):
    first_name= fake.first_name()
    last_name= fake.last_name()
    customer_record = {
        "customer_id": f"{customer_id+1000:04d}",
        "first_name": first_name,
        "last_name": last_name,
        "gender": random.choice(["Male", "Female"]),
        "dob": fake.date_of_birth(minimum_age=18, maximum_age=70).strftime("%Y-%m-%d"),
        "phone_number": custom_indian_phone_number(),
        "email": f'{first_name.lower()}.{last_name.lower()}@{fake.domain_name()}',
        "address": {
            "street": fake.street_address(),
            "city": fake.city(),
            "state": fake.state(),
            "zip": fake.postcode(),
            "country": "India"
        },
        "event_id": str(uuid.uuid4()),
        "event_time": datetime.utcnow().isoformat()
    }
    return customer_record


# -------------------
# Event Hub Producer
# -------------------
producer = EventHubProducerClient.from_connection_string(
    conn_str=CONNECTION_STR,
    eventhub_name=EVENTHUB_NAME
)

with producer:
    for i in range(50):  # generate and send 10 records
        record = generate_customer(i+1)
        event_data = EventData(json.dumps(record))
        producer.send_batch([event_data])
        print(f"✅ Sent: {record}")
        time.sleep(1)  # wait 1 sec between events
